# Load data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

prices = pd.read_csv('train.csv')
prices.head()



/home/lluis/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
# Read the training data
prices_all = pd.read_csv('train.csv', index_col='Id')

# First cleaning of rows with missing target and splitting training and validation
prices_all.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = prices_all.SalePrice
prices_all.drop(['SalePrice'], axis=1, inplace=True)

X_train_all, X_valid_all, y_train, y_valid = train_test_split(prices_all, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [3]:
#Automated selection of columns, either strings with low cardinality or numerical values

categorical_cols = [cname for cname in X_train_all.columns if
                    X_train_all[cname].nunique() < 10 and 
                    X_train_all[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_all.columns if 
                X_train_all[cname].dtype in ['int64', 'float64']]

relevant_cols = categorical_cols + numerical_cols
#relevant_cols

X_train = X_train_all[relevant_cols].copy()
X_valid = X_valid_all[relevant_cols].copy()

X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


# Training and Validation

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

## Preprocessing

In [5]:
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## Models

In [6]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [7]:
def RandomForest(*, n_estimators=100):
    return RandomForestRegressor(n_estimators=n_estimators, random_state=0)

def XGBoost(*, n_estimators=1000, learning_rate=0.05):
    return XGBRegressor(random_state=0, n_estimators = n_estimators, learning_rate = learning_rate)

In [8]:
n_RF = [10, 100, 500, 1000]
n_XGB = [100, 250, 500, 750]
LEARNING_XGB = [ 0.01, 0.4, 0.05, 0.06, 0.075, 0.1]

In [9]:
MODELS_RF = [ RandomForest(n_estimators=n) for n in n_RF ]
MODELS_XGB = [XGBoost(n_estimators=n, learning_rate=learning_rate) for n in n_XGB for learning_rate in LEARNING_XGB]

## Choosing best model

In [10]:
min_MAE = float("inf")
best_model = None
best_pipeline = None

In [11]:
# Bundle preprocessing and modeling code in a pipeline
for model in MODELS_RF + MODELS_XGB:
    local_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('model', model)
                                    ])

    # Preprocessing of training data, fit model 
    local_pipeline.fit(X_train, y_train)

    # Preprocessing of validation data, get predictions
    preds = local_pipeline.predict(X_valid)

    # Evaluate the model
    score = mean_absolute_error(y_valid, preds)
    print('MAE:', score)
    
    if( score < min_MAE ):
        min_MAE = score
        best_model = model
        best_pipeline = local_pipeline

print("Lowest MAE obtained: ", min_MAE, "from ", best_model)

MAE: 18931.834931506848
MAE: 17861.780102739725
MAE: 17406.934246575343
MAE: 17265.9764760274
MAE: 28399.426503638697
MAE: 17908.742133989726
MAE: 16980.710736836474
MAE: 17258.21813463185
MAE: 17405.436376284248
MAE: 17606.860913420376
MAE: 18520.884417808218
MAE: 17893.691085188355
MAE: 16725.72241545377
MAE: 17229.217024293663
MAE: 17407.40356110873
MAE: 17507.07102151113
MAE: 17231.49422089041
MAE: 17893.866679152397
MAE: 16719.48979291524
MAE: 17193.26614672517
MAE: 17432.928764447774
MAE: 17521.534393728594
MAE: 17060.80408550942
MAE: 17893.866679152397
MAE: 16718.94319884418
MAE: 17227.061403039384
MAE: 17432.4619140625
MAE: 17510.39327375856
Lowest MAE obtained:  16718.94319884418 from  XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=Non

# Prediction

In [12]:
X_test_full = pd.read_csv('test.csv', index_col='Id')
#Use same columns as before
X_test = X_test_full[relevant_cols].copy()

preds_test = best_pipeline.predict(X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)